In [11]:
from github import Github
import pandas as pd
import psycopg2
from datetime import datetime, timedelta
import os
import numpy as np

In [12]:
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
assert GITHUB_TOKEN, "GITHUBTOKEN environment variable not set!"
g = Github(GITHUB_TOKEN)

repo_names = [
    "meta-llama/llama3",
    "ollama/ollama",
    "langchain-ai/langgraph",
    "openai/openai-cookbook",
    "milvus-io/pymilvus"
]

C:\Users\yashv\AppData\Local\Temp\ipykernel_19872\988171778.py:3: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(GITHUB_TOKEN)


In [ ]:
end_date = datetime.now()
start_date = end_date - timedelta(days=60)
end_date_naive = end_date.replace(tzinfo=None)
start_date_naive = start_date.replace(tzinfo=None)
print("Start:", start_date_naive, "End:", end_date_naive)

Start: 2025-09-28 20:44:39.679739 End: 2025-11-27 20:44:39.679739


In [ ]:
conn = psycopg2.connect(
    dbname="spm_db",
    user="postgres",
    password="admin",
    host="localhost",
    port=5432
)
cur = conn.cursor()

In [ ]:
cur.execute("""
CREATE TABLE IF NOT EXISTS github_issues (
    repo VARCHAR(50),
    number INTEGER,
    title TEXT,
    state VARCHAR(10),
    created_at TIMESTAMP,
    closed_at TIMESTAMP,
    labels TEXT
);
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS github_repos (
    repo VARCHAR(50) PRIMARY KEY,
    stars INTEGER,
    forks INTEGER
);
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS github_pulls (
    repo VARCHAR(50),
    number INTEGER,
    title TEXT,
    state VARCHAR(10),
    created_at TIMESTAMP,
    merged_at TIMESTAMP,
    closed_at TIMESTAMP,
    user_login TEXT,
    labels TEXT
);
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS github_commits (
    repo VARCHAR(50),
    sha VARCHAR(50) PRIMARY KEY,
    author_login TEXT,
    committed_at TIMESTAMP,
    message TEXT
);
""")
conn.commit()
cur.close()

In [ ]:
cur = conn.cursor()
for repo_name in repo_names:
    repo = g.get_repo(repo_name)
    stars = repo.stargazers_count
    forks = repo.forks_count
    cur.execute("""
        INSERT INTO github_repos (repo, stars, forks)
        VALUES (%s, %s, %s)
        ON CONFLICT (repo) DO UPDATE SET stars = EXCLUDED.stars, forks = EXCLUDED.forks;
    """, (repo_name, stars, forks))
conn.commit()
cur.close()

In [ ]:
def fetch_issues(repo_name, start_datetime, end_datetime):
    repo = g.get_repo(repo_name)
    issues = repo.get_issues(state="all", since=start_datetime)
    data = []
    for issue in issues:
        created_at = issue.created_at.replace(tzinfo=None)
        if created_at > end_datetime.replace(tzinfo=None):
            continue
        data.append({
            "repo": repo_name,
            "number": issue.number,
            "title": issue.title,
            "state": issue.state,
            "created_at": issue.created_at.replace(tzinfo=None) if issue.created_at else None,
            "closed_at": issue.closed_at.replace(tzinfo=None) if issue.closed_at else None,
            "labels": ",".join([label.name for label in issue.labels])
        })
    return pd.DataFrame(data)

def insert_issues_csv(df, conn):
    cur = conn.cursor()
    for _, row in df.iterrows():
        closed_at = None if pd.isna(row.closed_at) else row.closed_at
        created_at = None if pd.isna(row.created_at) else row.created_at
        cur.execute("""
            INSERT INTO github_issues
            (repo, number, title, state, created_at, closed_at, labels)
            VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, (row.repo, row.number, row.title, row.state, created_at, closed_at, row.labels))
    conn.commit()
    cur.close()

# For each repo, fetch and insert
for repo_name in repo_names:
    df = fetch_issues(repo_name, start_date_naive, end_date_naive)
    insert_issues_csv(df, conn)
    print(f"Inserted {len(df)} issues for {repo_name}")

Inserted 6 issues for meta-llama/llama3
Inserted 1316 issues for ollama/ollama
Inserted 372 issues for langchain-ai/langgraph
Inserted 184 issues for openai/openai-cookbook
Inserted 132 issues for milvus-io/pymilvus


In [ ]:
cur = conn.cursor()
for repo_name in repo_names:
    repo = g.get_repo(repo_name)
    pulls = repo.get_pulls(state="all")
    for pr in pulls:
        cur.execute("""
            INSERT INTO github_pulls (repo, number, title, state, created_at, merged_at, closed_at, user_login, labels)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT DO NOTHING;
        """, (
            repo_name,
            pr.number,
            pr.title,
            pr.state,
            pr.created_at,
            getattr(pr, 'merged_at', None),
            pr.closed_at,
            pr.user.login if pr.user else None,
            ','.join([l.name for l in pr.labels])
        ))
conn.commit()
cur.close()

In [ ]:
cur = conn.cursor()
for repo_name in repo_names:
    repo = g.get_repo(repo_name)
    commits = repo.get_commits()
    for c in commits:
        sha = c.sha
        author_login = None
        if getattr(c, "author", None):
            author_login = c.author.login
        elif getattr(c, "commit", None) and getattr(c.commit, "author", None):
            author_login = getattr(c.commit.author, "name", None)
        committed_at = None
        if getattr(c, "commit", None):
            if getattr(c.commit, "committer", None) and getattr(c.commit.committer, "date", None):
                committed_at = c.commit.committer.date
            elif getattr(c.commit, "author", None) and getattr(c.commit.author, "date", None):
                committed_at = c.commit.author.date
        message = c.commit.message if getattr(c, "commit", None) else None
        cur.execute("""
            INSERT INTO github_commits (repo, sha, author_login, committed_at, message)
            VALUES (%s, %s, %s, %s, %s)
            ON CONFLICT (sha) DO NOTHING;
        """, (
            repo_name,
            sha,
            author_login,
            committed_at,
            message
        ))
conn.commit()
cur.close()